In [1]:
import numpy as np
import torch
import Doc_import
import os
import json
import Data_Preprocessing

In [2]:
train_features, train_labels, test_features, test_labels = Doc_import.import_pan21(base_path=r"C:\Users\jan-p\Documents\Uni Wuppertal\Semester 2\Information "
                                 r"Retrival\Projekt\Data\pan21", returns= 100)


In [3]:
len(train_features)

100

In [4]:
def preprocessing_task1_binary(labels):
    new_labels = list()
    binary = list()
    count_ja = 0
    count_nein = 0
    for label in labels:
        if label['multi-author'] == 1:
            binary.append(1)
            new_labels.append([1,0])
            count_ja += 1
        else:
            binary.append(0)
            new_labels.append([0,1])
            count_nein += 1
    return new_labels,binary, count_ja, count_nein

In [7]:
labels,bin,  ja, nein = preprocessing_task1_binary(train_labels)
print(ja, nein)
labels

82 18


[[1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 0],
 [1, 0]]

In [6]:
'''ja = list()
nein = list()
for i, b in enumerate(bin):
    if b == 1:
        ja.append(i)
    else: nein.append(i)
ja = ja[:2800]

import random
random.shuffle(ja)
random.shuffle(nein)

train_labels_new = list()
train_features_new = list()

for i in range(2800):
    train_labels_new.append(labels[nein[i]])
    train_labels_new.append(labels[ja[i]])
    train_features_new.append(train_features[nein[i]])
    train_features_new.append(train_features[ja[i]])
train_labels_new'''

'ja = list()\nnein = list()\nfor i, b in enumerate(bin):\n    if b == 1:\n        ja.append(i)\n    else: nein.append(i)\nja = ja[:2800]\n\nimport random\nrandom.shuffle(ja)\nrandom.shuffle(nein)\n\ntrain_labels_new = list()\ntrain_features_new = list()\n\nfor i in range(2800):\n    train_labels_new.append(labels[nein[i]])\n    train_labels_new.append(labels[ja[i]])\n    train_features_new.append(train_features[nein[i]])\n    train_features_new.append(train_features[ja[i]])\ntrain_labels_new'

In [10]:
from transformers import BertModel

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import Data_Preprocessing

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        kernel = 4
        input_ff = int(393216/(kernel * kernel))
        zwischenlayer1 = int(1000)
        zwischenlayer2 = int(10)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.pooling = nn.MaxPool2d(kernel)
        self.linear11 = nn.Linear(input_ff, zwischenlayer1)
        self.linear12 = nn.Linear(zwischenlayer1, zwischenlayer2)
        self.linear13 = nn.Linear(zwischenlayer2, 2)

    def forward(self, input_data):
        bert_output = self.bert(**input_data)
        bert_output = self.pooling(bert_output[0])
        bert_output = bert_output[0].view(-1, )
        x1 = torch.sigmoid(self.linear11(bert_output))
        x1 = torch.sigmoid(self.linear12(x1))
        x1 = torch.sigmoid(self.linear13(x1))

        return x1

    def train_x1(self, features, labels, loss_function, optimizer, epochs):
        log_interval = 5
        evaluation_list = list()
        for epoch in range(epochs):
            for i, data in enumerate(features):
                data = Data_Preprocessing.preprocessing(data)
                optimizer.zero_grad()
                output1 = self.forward(data)
                target =  torch.tensor(labels[i])
                target = target.to(torch.float32)
                #print(torch.tensor(output1[0][0]),torch.tensor(float(labels[i]['multi-author'])))
                print(output1[0],target)
                #loss_x1 = loss_function(torch.tensor(output1[0][0]), torch.tensor(float(labels[i]['multi-author'])))
                loss_x1 = loss_function(output1[0],target)
                evaluation_list.append(([output1,torch.tensor(labels[i]), loss_x1]))
                print(i, '  ist=', output1, ' soll=', torch.tensor(labels[i]), ' Loss=', loss_x1)
                loss_x1.backward()
                optimizer.step()
                #if index % log_interval == 0:
                 #   print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epochs, index * len(features),
                  #                                                                 len(features),
                   #                                                                100. * index / len(features),
                    #                                                               loss_x1.data.item()))
        return evaluation_list

    def test_model(self, features, labels, loss_function):
        log_interval = 5
        evaluation_list = list()

        for index, data in enumerate(features):
            output = self(data)
            target =  torch.tensor(labels[index])
            target = target.to(torch.float32)
            loss = loss_function(output[0], target)
            evaluation_list.append(([output,torch.tensor(labels[index]['multi-author']), loss]))



        return evaluation_list

In [11]:
model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.BCELoss()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
model.train()
evaluation = model.train_x1(features=train_features,labels=labels ,loss_function= criterion,optimizer= optimizer,epochs= 1)

C:\Users\jan-p\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([0.4549, 0.5590], grad_fn=<SigmoidBackward0>) tensor([1., 0.])


ValueError: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.

In [ ]:
evaluation